In [5]:
import efel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy

In [6]:
# define some functions
def find_spike_times(voltages, dt, detection_level, min_interval):
    spike_times = []
    last_spike_time = -min_interval
    for i in range(1, len(voltages)):
        t = i * dt
        if (voltages[i - 1] < detection_level <= voltages[i]) and (t - last_spike_time >= min_interval):
            spike_times.append(t)
            last_spike_time = t
    return spike_times


def group_cvs(values, group_size):
    cvs = []
    if len(values) >= group_size:
        for i in range(0, len(values) - group_size, group_size):
            mu = np.mean(values[i:i + group_size])
            sigma = np.std(values[i:i + group_size])
            cvs.append(sigma/mu)
    return cvs

def segment(values, dx, x_min, x_max):
    return values[round(x_min / dx):round(x_max / dx)]

def find_slopes(values, dx):
    diffs = np.diff(values)
    slopes = [0] * len(values)
    slopes[0] = diffs[0] / dx
    slopes[-1] = diffs[-1] / dx
    for i in range(1, len(values) - 1):
        slopes[i] = (diffs[i - 1] + diffs[i]) / (2 * dx)
    return (slopes)

#use neo to import either voltage or current clamp data in the correct, scaled units!
def load_neo_file(file_name, **kwargs):
    import neo
    reader = neo.io.get_io(file_name)
    blocks = reader.read(**kwargs)
    new_blocks = []
    for bl in blocks:
        new_segments = []
        for seg in bl.segments:
            traces = []
            count_traces = 0
            analogsignals = seg.analogsignals

            for sig in analogsignals:
                traces.append({})
                traces[count_traces]['T'] = sig.times.rescale('ms').magnitude
                #need to write an if statement here for conversion
                try:
                    traces[count_traces]['A'] = sig.rescale('pA').magnitude
                except:
                    traces[count_traces]['V'] = sig.rescale('mV').magnitude
                count_traces += 1
            new_segments.append(traces)
        #new_blocks.append(efel_segments)
    return new_segments

def dvdt(path, sweep):
    table2 = []
    os.chdir(path)
    for filename in os.listdir():
        #check whether file is in the axgx or axgd format
        if filename.endswith(".axgd") or filename.endswith(".axgx"):
            [traces] = efel.io.load_neo_file(filename, stim_start=500, stim_end=2500)
            for data in traces[sweep]:
                times = (data['T']) / 1000
                voltages = (data['V'])
                times -= times[0]
                dt = times[2] - times[1]
                detection_level = 0
                min_interval = 0.0001
                spike_times = find_spike_times(voltages, dt, detection_level, min_interval)
                isis = np.diff(spike_times)
                time_before = .018
                time_after = .015
                times_rel = list(np.arange(-time_before, time_after, dt))
                spike_voltages = []
                for i in range(0, len(spike_times)):
                    if time_before < spike_times[i] < times[-1] - time_after:
                        spike_voltages.append(
                            segment(voltages, dt, spike_times[i] - time_before, spike_times[i] + time_after))
                spike_voltage_arrays = [np.array(x) for x in spike_voltages]
                mean_spike_voltages = [np.mean(k) for k in zip(*spike_voltage_arrays)]
                dvdt_threshold = 20
                dvdt = find_slopes(mean_spike_voltages, dt)
                i = 1
                while dvdt[i] < dvdt_threshold:
                    i += 1
                v0 = mean_spike_voltages[i - 1]
                v1 = mean_spike_voltages[i]
                dvdt0 = dvdt[i - 1]
                dvdt1 = dvdt[i]
                v_threshold = v0 + (v1 - v0) * (dvdt_threshold - dvdt0) / (dvdt1 - dvdt0)
                pandas_dvdt = pd.DataFrame(dvdt)
                pandas_dvdt.rename(columns={0:filename}, inplace=True) #naming the columns!
                pandas_membrane_voltages = pd.DataFrame(mean_spike_voltages)
            table2.append(pandas_dvdt)
            df_concat = pd.concat(table2, axis=1)

            df_concat.to_excel('dvdt' + 'master_file.xlsx', index=False)
    return(df_concat)

#write some functions for the rest of this stuff
def analyze_feature(path, feature):
    table2 = []
    os.chdir(path)
    for filename in os.listdir():
        table = pd.DataFrame(columns=[feature])     #create a table that has columns with the name you want
        table.name = feature                        #the tables name
        if filename.endswith(".axgd") or filename.endswith(".axgx"):    #check for the filetype
            [traces] = efel.io.load_neo_file(filename, stim_start=500, stim_end=2500)    #load the trace, and define stim start and stop
            for data in traces:    #loop through these guys
                #table.rename(columns={feature:filename}, inplace=True) #renaming the columns with the correct file !
                feature_values = efel.getFeatureValues(data, [feature], raise_warnings=None)[0]  #this is the feature extraction
                if feature_values[feature] is not None:
                    # Define the parameters for detection
                    efel.api.setThreshold(-10) # Voltage threshold for detection
                    efel.api.setDerivativeThreshold(20) # dV/dt threshold for detection
                    efel.setIntSetting('strict_stiminterval', True)
                    length = len(table)
                    table.loc[length, feature] = feature_values[feature][0]

                else:
                    efel.api.setThreshold(-10) # Voltage threshold for detection
                    efel.api.setDerivativeThreshold(20) # dV/dt threshold for detection
                    efel.setIntSetting('strict_stiminterval', True)
                    length = len(table)
                    table.loc[length, feature] = feature_values[feature]

            table2.append(table)
            df_concat = pd.concat(table2, axis=1)
            table.rename(columns={feature:filename}, inplace=True) #renaming the columns with the correct file !
            #block of code to combine all of the generated excel workbooks into a single workbook
            df_concat.to_excel(feature + 'master_file.xlsx', index=False)
    Current_injected = np.linspace(-100.0, 600.0, num=15)
    table2 = df_concat.assign(Current_injected=Current_injected)
    #lineplot = df_concat.plot()
    #sns_lineplot = sns.relplot(data = table2, x = "Spikecount_stimint", y = 'Current_injected', kind="line")
    return(df_concat)


def mean_spike_voltages(path, sweep):
    table2 = []
    os.chdir(path)
    for filename in os.listdir():
        #check whether file is in the axgx or axgd format
        if filename.endswith(".axgd") or filename.endswith(".axgx"):
            [traces] = efel.io.load_neo_file(filename, stim_start=500, stim_end=2500)
            for data in traces[sweep]:
                times = (data['T'])/1000
                voltages = (data['V'])
                times -= times[0]
                dt = times[2] - times[1]
                detection_level = 0
                min_interval = 0.0001
                spike_times = find_spike_times(voltages, dt, detection_level, min_interval)
                isis = np.diff(spike_times)
                time_before = .018
                time_after = .015
                times_rel = list(np.arange(-time_before, time_after, dt))
                spike_voltages = []
                for i in range(0, len(spike_times)):
                    if time_before < spike_times[i] < times[-1] - time_after:
                        spike_voltages.append(segment(voltages, dt, spike_times[i] - time_before, spike_times[i] + time_after))
                spike_voltage_arrays = [np.array(x) for x in spike_voltages]
                mean_spike_voltages = [np.mean(k) for k in zip(*spike_voltage_arrays)]
                dvdt_threshold = 20
                dvdt = find_slopes(mean_spike_voltages, dt)
                i = 1
                while dvdt[i] < dvdt_threshold:
                    i += 1
                v0 = mean_spike_voltages[i - 1]
                v1 = mean_spike_voltages[i]
                dvdt0 = dvdt[i - 1]
                dvdt1 = dvdt[i]
                v_threshold = v0 + (v1 - v0) * (dvdt_threshold - dvdt0) / (dvdt1 - dvdt0)
                pandas_dvdt = pd.DataFrame(dvdt)
                pandas_membrane_voltages = pd.DataFrame(mean_spike_voltages)
                pandas_membrane_voltages.rename(columns={0:filename}, inplace=True)

            table2.append(pandas_membrane_voltages)
            df_concat = pd.concat(table2, axis=1)
            pandas_times_rel = pd.DataFrame(times_rel)
            df_concat.to_excel('mean_spike_voltages' + 'master_file.xlsx', index=False)
            pandas_times_rel.to_excel('times_rel_for_VTA.xlsx', index=False)
    return(df_concat, pandas_times_rel)

def A_current(path):
    os.chdir(path)
    table2 = []
    table3 = []
    table4 = []
    for file_name in os.listdir():
        if file_name.endswith(".axgd") or file_name.endswith(".axgx"):
            file = load_neo_file(file_name)
            def monoExp(x, m, t, b):
                return m * np.exp(-t * x) + b

            for traces in file:
                for data in traces:
                    #this is for the first part of the A-current
                    times1 = data['T'][10050:20000]
                    amps1 = data['A'][10050:20000]
                    #plt.plot(times, amps) #inspect
                    baseline = np.mean(data['A'][:3000])  #define a baseline period from which to substract from
                    amps1 = amps1 - baseline  #subtract the baseline from the amplitudes of the first pulse
                    max_amp1 = np.amax(amps1)  #find the maximum of the baseline subtracted amplitudes
                    #working on integration
                    flat_amps_1 = np.ndarray.flatten(
                        amps1)  #here we have to flatten our array into a single dimension so we can integrate
                    integrate1 = np.trapz(flat_amps_1)  #lets integrate this function using the numpy trapezius function
                    #this is for the substraction of the next part, so that we can get the substracted information
                    times2 = data['T'][
                             35050:40000]  #in order to extract a-current, we need to subtract from the inactivated standing current
                    amps2 = data['A'][
                            35050:40000]  #in order to extract a-current, we need to subtract from the inactivated standing current
                    #plt.plot(times2, amps2) #inspect
                    amps2 = amps2 - baseline  #find the baseline subtracted values for the inactivated standing current
                    max_amp2 = np.amax(amps2)  #find the max amplitude of that current
                    flat_amps_2 = np.ndarray.flatten(
                        amps2)  #flatten the baseline substracted inactivated portion for integration
                    integrate2 = np.trapz(flat_amps_2)  #integrate flattened baseline substracted sweeps
                    #now we need to substract that information to get the A current values
                    a_current_max_amp = max_amp1 - max_amp2  #this and the next calls are for the actual data we want, here max amp
                    a_current_auc = integrate1 - integrate2  #and here AUC
                    #from this point foward we are trying to put data together for iteration
                    #print('A-Current Max Amp is ', a_current_max_amp, 'pA')
                    #print('A-Current AUC is', a_current_auc, 'pA*s')
                    #generate a dataframe, must pass a 2d array
                    a_current_max_amp_array = np.array(a_current_max_amp, ndmin=2)
                    max_amp_df = pd.DataFrame(a_current_max_amp_array)
                    max_amp_df['file_name'] = file_name  #adding a column to add the filename

                    #generate a dataframe, must mass a 2d array
                    a_current_auc_array = np.array(a_current_auc, ndmin=2)
                    auc_df = pd.DataFrame(a_current_auc_array)
                    auc_df['file_name'] = file_name  #adding a column to add the filename

                    #fit a curve using scipy functionality - these params seem to work for a-current
                    p0 = [500, .001, 50]  #values near what we expect   #here
                    params, cv = scipy.optimize.curve_fit(monoExp, times1, flat_amps_1, p0, bounds=(-np.inf, np.inf), maxfev=50000)  #here
                    m, t, b = params  #here
                    #m, t = params
                    sampleRate = 10_000  #hz
                    tauSec = (1 / t) / sampleRate
                    #determine quality of fit
                    squaredDiffs = np.square(flat_amps_1 - monoExp(times1, m, t, b))  #here
                    squaredDiffsFromMean = np.square(flat_amps_1 - np.mean(flat_amps_1))
                    rSquared = 1 - np.sum(squaredDiffs) / np.sum(
                        squaredDiffsFromMean)  #we want these, but they arent super important to display
                    #print(f"R^2 = {rSquared}")

                    #plot results
                    #plt.plot(times1, flat_amps_1, '.', label="data")
                    #plt.plot(times1, monoExp(times1, m, t, b), '--', label="fitted")  #here
                    #plt.show()
                    #plt.title("Fitted Expotential Curve")

                    #inspect the params
                    #print(f"Y = {m} * e^(-{t} * x) + {b}")   #the equations are important
                    #print(f"Tau = {tauSec * 1e6} us")    #but the tau is the most important
                    tau_array = np.array(tauSec * 1e4, ndmin=2)

                    tau_df = pd.DataFrame(tau_array)
                    tau_df['file_name'] = file_name  #adding a column to add the filename
                table2.append(max_amp_df)

                table3.append(auc_df)
                table4.append(tau_df)

        amp_concat = pd.concat(table2, ignore_index=True, axis=0)
        amp_concat.rename(columns = {0:'Control Max Amplitude(pA)'}, inplace=True)
        #amp_concat['Sweep'] = amp_concat.index%5 + 1
        every_4th_sweep_amp = amp_concat[amp_concat.index % 5 == 3]
        every_4th_sweep_amp.to_excel('a_current_amp' + '.xlsx', index=False)

        auc_concat = pd.concat(table3, ignore_index=True, axis=0)
        auc_concat.rename(columns = {0:'Control AUC (pA*s)'}, inplace=True)
        #auc_concat['Sweep'] = amp_concat.index%5 + 1
        every_4th_sweep_auc = auc_concat[amp_concat.index % 5 == 3]
        every_4th_sweep_auc.to_excel('a_current_auc' + '.xlsx', index=False)

        tau_concat = pd.concat(table4, ignore_index=True, axis=0)
        tau_concat.rename(columns = {0:'Control Tau (ms)'}, inplace=True)
        #tau_concat['Sweep'] = amp_concat.index%5 + 1
        every_4th_sweep_tau = tau_concat[amp_concat.index % 5 == 3]
        every_4th_sweep_tau.to_excel('a_current_tau' + '.xlsx', index=False)
    return display(every_4th_sweep_amp), display(every_4th_sweep_auc), display(every_4th_sweep_tau)

def sk_analysis(path):

    def monoExp(x, m, t, b):
        return m * np.exp(-t * x) + b

    ahc_append = []
    ahc_auc_append = []

    ahc_max_amp_append = []
    auc_append = []
    tau_append = []
    ahc_max_amp_df_append = []
    os.chdir(path)

    for file_name in os.listdir():
        tabla = []
        if file_name.endswith(".axgd") or file_name.endswith(".axgx"):
            traces = load_neo_file(file_name)
            for sk_data in traces:
                for data in sk_data:
                    #this is for the first part of the A-current
                    times1 = data['T'][322002:324260]  #this is 16.101 to 16.213
                    amps1 = data['A'][322002:324260]
                    #plt.plot(times, amps) #inspect
                    baseline = np.mean(data['A'][318400:319600])  #define a baseline period from which to substract from
                    amps1 = amps1 - baseline  #subtract the baseline from the amplitudes of the first pulse
                    amps1_df = pd.DataFrame(amps1) #we generated all the amps into a dataframe, check
                    flat_times = np.ndarray.flatten(times1) #we have all of the times into a flattened numpy array, check
                    tabla.append(amps1_df) #and we appended all of the amps into a dataframe
                amps_concat = pd.concat(tabla, axis=1) #this is correct - we put all the amps for a given trace into a dataframe
            #now we need to average those dataframes by row
            averaged_sk_trace = amps_concat.mean(axis=1)  #woohooo this is the averaged SK trace in a pd.DF, this is what we want to work with from here on out!

            #Block of code of AHC Max Amp !
            ahc_max_amp = pd.DataFrame.max(averaged_sk_trace)  #max of the whole ahc, not just sk component, This is the value!!
            #figure out to how put this in a format that can be read and concatenated
            ahc_max_amp_array = np.array(ahc_max_amp, ndmin=2)   #these lines are new
            ahc_max_amp_df = pd.DataFrame(ahc_max_amp_array)  #these lines are new - this line of code produces the correct values but does not add them into an appended dataframe. which is obviously a problem. the task is to get these and all the values from the other two measures into a single dataframe so that in the future we may compare across groups within python<3
            ahc_max_amp_df['file_name'] = file_name  #adding a column to add the filename
            ahc_append.append(ahc_max_amp_df) #Here!
            ahc_max_amp_concat_df = pd.concat(ahc_append) #This is the variable for ahc

            #Block of code for AHC AUC
            averaged_sk_trace_as_np = averaged_sk_trace.to_numpy()
            flattened_average_sk_trace = np.ndarray.flatten(averaged_sk_trace_as_np)
            ahc_auc = np.trapz(flattened_average_sk_trace)  #this is the auc of the whole ahc
            #bit of code to get the ahc auc into readable condition
            ahc_auc_array = np.array(ahc_auc, ndmin=2)
            ahc_auc_df = pd.DataFrame(ahc_auc_array)  #here we have the auc data in a dataframe
            ahc_auc_df['file_name'] = file_name
            ahc_auc_append.append(ahc_auc_df)
            ahc_auc_concat_df = pd.concat(ahc_auc_append) #this is the variable for auc

            #Block of code for kinetics
            trace_for_kinetics = flattened_average_sk_trace[30:]
            times_for_kinetics = flat_times[30:]
            trace_for_kinetics_pd = pd.DataFrame(trace_for_kinetics)
            #trace_for_kinetics_pd.to_excel("trace_for_kinetics_pd.xlsx")
            times_for_kinetics_pd = pd.DataFrame(times_for_kinetics)
            #times_for_kinetics_pd.to_excel("times_for_kinetics_pd.xlsx")

            #fit the curve for inactivation tau
            p0 = [1.187 * 10 ** 150., .02, +16]  #values near what we expect   #here
            params, cv = scipy.optimize.curve_fit(monoExp, times_for_kinetics, trace_for_kinetics, p0,
                                                  bounds=(-np.inf, np.inf),
                                                  maxfev=100000)  #here  #this fits the training curve with an r-squared of 0.97
            m, t, b = params  #here
            #m, t = params
            sampleRate = 20_000  #hz
            tauSec = (1 / t) / sampleRate

            #determine quality of fit
            squaredDiffs = np.square(trace_for_kinetics - monoExp(times_for_kinetics, m, t, b))  #here
            squaredDiffsFromMean = np.square(trace_for_kinetics - np.mean(trace_for_kinetics))
            rSquared = 1 - np.sum(squaredDiffs) / np.sum(
                squaredDiffsFromMean)  #we want these, but they arent super important to display
            #print(f"R^2 = {rSquared}")

            #plot results
            #plt.plot(times_for_kinetics, trace_for_kinetics, '.', label="data")
            #plt.plot(times_for_kinetics, monoExp(times_for_kinetics, m, t, b), '--', label="fitted")  #here
            plt.show()
            #plt.title("Fitted Expotential Curve")

            #inspect the params
            #print(f"Y = {m} * e^(-{t} * x) + {b}")   #the equations are important
            #print(f"Tau = {tauSec * 1e6} us")    #but the tau is the most important
            plt.show()
            tau_flat_ms = tauSec * 1e4

            #Bit of code to get tau into working order
            tau_array = np.array(tauSec * 1e4, ndmin=2)
            tau_df = pd.DataFrame(tau_array)
            tau_df['file_name'] = file_name
            tau_append.append(tau_df)
            tau_concat_df = pd.concat(tau_append)   #this is the variable for tau

    #lets rename columns and export to excel for each of our metrics
    ahc_max_amp_concat_df.rename(columns = {0:'Control AHC Max_Amplitude (pA)'}, inplace=True)
    ahc_max_amp_concat_df.to_excel('ahc_max_amp_' + '.xlsx', index=False)

    ahc_auc_concat_df.rename(columns = {0:'Control AHC AUC (pA*s)'}, inplace=True)
    ahc_auc_concat_df.to_excel('ahc_auc' + '.xlsx', index=False)

    tau_concat_df.rename(columns = {0:'Control Decay Tau (ms)'}, inplace=True)
    tau_concat_df.to_excel('ahc_tau' + '.xlsx', index=False)

    return display(ahc_max_amp_concat_df), display(ahc_auc_concat_df), display(tau_concat_df)

def generalized_v_clamp_analysis(path, sampling_rate_hz, analysis_time_start_ms, analysis_time_end_ms, baseline_start_ms, baseline_end_ms, time_from_analysis_start_time_for_decay_tau):
    os.chdir(path)
    ahc_append = []
    ahc_auc_append = []
    tau_append = []

    def monoExp(x, m, t, b):
        return m * np.exp(-t * x) + b
    #here we need to convert sampling rate to time in ms, and as integers, otherwise we cannot slice with them
    analysis_time_start1 = int(sampling_rate_hz * analysis_time_start_ms)
    analysis_time_end1 = int(sampling_rate_hz * analysis_time_end_ms)
    baseline_start1 = int(sampling_rate_hz * baseline_start_ms)
    baseline_end1 = int(sampling_rate_hz * baseline_end_ms)
    time_from_analysis_start_time_for_decay_tau1 = int(sampling_rate_hz * time_from_analysis_start_time_for_decay_tau)
    for file_name in os.listdir():
        tabla = []
        if file_name.endswith(".axgd") or file_name.endswith(".axgx"):
            traces = load_neo_file(file_name)
            for sk_data in traces:
                for data in sk_data:
                    #this is for the first part of the A-current
                    #time1_slice = slice(analysis_time_start1:analysis_time_end1)
                    times1 = data['T'][analysis_time_start1:analysis_time_end1]  #this is 16.101 to 16.213
                    amps1 = data['A'][analysis_time_start1:analysis_time_end1]
                    #plt.plot(times, amps) #inspect
                    baseline = np.mean(data['A'][baseline_start1:baseline_end1])  #define a baseline period from which to substract from
                    amps1 = amps1 - baseline  #subtract the baseline from the amplitudes of the first pulse
                    amps1_df = pd.DataFrame(amps1)
                    flat_times = np.ndarray.flatten(times1)
                    tabla.append(amps1_df)
                amps_concat = pd.concat(tabla, axis=1)
            averaged_sk_trace = amps_concat.mean(axis=1)  #woohooo this is the averaged SK trace in a pd.DF

            #Block of code for AHC Max Amplitude in pA
            ahc_max_amp = pd.DataFrame.max(averaged_sk_trace)  #max of the whole ahc, not just sk component
            ahc_max_amp_array = np.array(ahc_max_amp, ndmin=2)   #these lines are new
            ahc_max_amp_df = pd.DataFrame(ahc_max_amp_array)  #these lines are new
            ahc_max_amp_df['file_name'] = file_name
            ahc_append.append(ahc_max_amp_df) #Here!
            ahc_max_amp_concat_df = pd.concat(ahc_append)

            #Block of code for AUC
            averaged_sk_trace_as_np = averaged_sk_trace.to_numpy()
            flattened_average_sk_trace = np.ndarray.flatten(averaged_sk_trace_as_np)
            ahc_auc = np.trapz(flattened_average_sk_trace)  #this is the auc of the whole ahc
            #bit of code to get the ahc auc into readable condition
            ahc_auc_array = np.array(ahc_auc, ndmin=2)
            ahc_auc_df = pd.DataFrame(ahc_auc_array)  #here we have the auc data in a dataframe
            ahc_auc_df['file_name'] = file_name
            ahc_auc_append.append(ahc_auc_df)
            ahc_auc_concat_df = pd.concat(ahc_auc_append)

            #Block of code for decay tau
            trace_for_kinetics = flattened_average_sk_trace[time_from_analysis_start_time_for_decay_tau1:]
            times_for_kinetics = flat_times[time_from_analysis_start_time_for_decay_tau1:]
            trace_for_kinetics_pd = pd.DataFrame(trace_for_kinetics)
            #trace_for_kinetics_pd.to_excel("trace_for_kinetics_pd.xlsx")
            times_for_kinetics_pd = pd.DataFrame(times_for_kinetics)
            #times_for_kinetics_pd.to_excel("times_for_kinetics_pd.xlsx")

            #fit the curve for inactivation tau
            p0 = [1.187 * 10 ** 150., .02, +16]  #values near what we expect   #here
            params, cv = scipy.optimize.curve_fit(monoExp, times_for_kinetics, trace_for_kinetics, p0,
                                                  bounds=(-np.inf, np.inf),
                                                  maxfev=100000)  #here  #this fits the training curve with an r-squared of 0.97
            m, t, b = params  #here
            #m, t = params
            sampleRate = sampling_rate_hz  #hz
            tauSec = (1 / t) / sampleRate

            #determine quality of fit
            squaredDiffs = np.square(trace_for_kinetics - monoExp(times_for_kinetics, m, t, b))  #here
            squaredDiffsFromMean = np.square(trace_for_kinetics - np.mean(trace_for_kinetics))
            rSquared = 1 - np.sum(squaredDiffs) / np.sum(
                squaredDiffsFromMean)  #we want these, but they arent super important to display
            #print(f"R^2 = {rSquared}")

            #plot results
            #plt.plot(times_for_kinetics, trace_for_kinetics, '.', label="data")
            #plt.plot(times_for_kinetics, monoExp(times_for_kinetics, m, t, b), '--', label="fitted")  #here
            plt.show()
            #plt.title("Fitted Expotential Curve")

            #inspect the params
            #print(f"Y = {m} * e^(-{t} * x) + {b}")   #the equations are important
            #print(f"Tau = {tauSec * 1e6} us")    #but the tau is the most important
            plt.show()
            tau_flat_ms = tauSec * 1e4
            tau_array = np.array(tauSec * 1e4, ndmin=2)
            tau_df = pd.DataFrame(tau_array)
            tau_df['file_name'] = file_name
            tau_append.append(tau_df)
            tau_concat_df = pd.concat(tau_append)
    #for file_name_1 in os.listdir():
    #if file_name_1.endswith(".axgd") or file_name_1.endswith(".axgx"):
    #filename = file_name_1

    #Block of code to add a column containing the file name for each metric, change the column name, and print to excel
    #ahc_max_amp_concat_df['File'] = filename
    ahc_max_amp_concat_df.rename(columns = {0:'Max Amplitude(pA)'}, inplace=True)
    ahc_max_amp_concat_df.to_excel('Gen_'+'ahc_max_amp' + '.xlsx', index=False)
    #bit of tricky code for the auc

    #ahc_auc_concat_df['File'] = filename
    ahc_auc_concat_df.rename(columns = {0:'AUC (pA*s)'}, inplace=True)
    ahc_auc_concat_df.to_excel('Gen_'+'ahc_auc' + '.xlsx', index=False)

    #same bit for the decay tau
    #tau_concat_df['File'] = filename
    tau_concat_df.rename(columns = {0:'Tau (ms)'}, inplace=True)
    tau_concat_df.to_excel('Gen_'+'ahc_tau' + '.xlsx', index=False)
    return display(ahc_max_amp_concat_df), display(ahc_auc_concat_df), display(tau_concat_df)

def AHP(path, sweep):
    ahp_array = []
    os.chdir(path)
    for file_name in os.listdir():

        if file_name.endswith(".axgd") or file_name.endswith(".axgx"):
            file = load_neo_file(file_name)
            def monoExp(x, m, t, b):
                return m * np.exp(-t * x) + b
            for traces in file:
                for data in traces:
                    #this is for the first part of the A-current
                    times1 = data['T'][25000:30000]
                    volts1 = data['V'][25000:30000]
                    baseline = data['V'][4500:4910]
                    min_volts = np.min(volts1)
                    ahp_min = min_volts - baseline
                    max_ahp = ahp_min[sweep]
            ahp_array.append(max_ahp)
    ahp_df = pd.DataFrame(ahp_array)
    ahp_df.to_excel('AHP_min' + '.xlsx', index=False)
    #ahp_concat = pd.concat(ahp_array, ignore_index=True, axis=0)
    return display(ahp_df)

In [ ]:
cc_path =
os.chdir(cc_path)
analyze_feature(cc_path, feature='ISI_CV')
analyze_feature(cc_path, feature='time_to_first_spike')
analyze_feature(cc_path, feature='Spikecount_stimint')
analyze_feature(cc_path, feature='mean_frequency')
analyze_feature(cc_path, feature='steady_state_voltage_stimend')
analyze_feature(cc_path, feature='doublet_ISI')
analyze_feature(cc_path, feature='amp_drop_first_last')
analyze_feature(cc_path, feature='spike_half_width')

In [ ]:
A_current_path =
A_current(A_current_path)

In [ ]:
ahc_path =
sk_analysis(ahc_path)